In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy import ndimage
from func import *
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
from astropy.table import Table, hstack
import os
import glob
from astropy.table import Table, vstack, join
import pandas as pd
from scipy.optimize import leastsq


In [2]:
def scale_factor_spectrum(z,data,error,wave):
    k = 1+z
    select1 = (wave>5030*k) & (wave<5050*k)
    cont_dat = data[select1]
    cont_err = error[select1]
    error_maincube = np.std(cont_dat)
    error_errcube = np.mean(cont_err)
    fact = error_maincube/error_errcube
    return fact

def aperture_spec_fit(spectrum,err,mini_wave,p_init,broad2=False,MC_loops=100):
    if broad2:
        full_gauss = full_gauss2
    else:
        full_gauss = full_gauss1
    popt_full_fit,pcov_full_fit = leastsq(full_gauss,x0=p_init,args=(mini_wave,spectrum,err),maxfev = 10000000)       
    fitted=(full_gauss(popt_full_fit,mini_wave,spectrum,err))*(err)+spectrum 
    residual = spectrum - fitted
    v_out = [popt_full_fit[2] - popt_full_fit[6]]
    parameters = np.append(popt_full_fit,v_out)
    return parameters

def flux_blr(amp_Hb1,amp_Hb2,sigma_eff_Hb1,sigma_eff_Hb2,broad2=False):
    flux1 = amp_Hb1*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb1))
    flux2 = amp_Hb2*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb2)) 
    if broad2:
        flux = (flux1 + flux2)*(10**(-16))
    else:
        flux = flux1*(10**(-16)) 
    return flux

def flux_FeII_5018(amp_FeII_5018_1,amp_FeII_5018_2,sigma_eff_Hb1,sigma_eff_Hb2,broad2=False):
    flux1 = amp_FeII_5018_1*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb1))
    flux2 = amp_FeII_5018_2*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb2)) 
    if broad2:
        flux = (flux1 + flux2)*(10**(-16))
    else:
        flux = flux1*(10**(-16)) 
    return flux

def Hb_blr_lum(z,flux):
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
    D_L = cosmo.luminosity_distance(z)
    dist = D_L.to(u.cm)
    Hb_lum = np.log10(4*np.pi)+2*np.log10(dist.value)+np.log10(flux)
    return Hb_lum
    
def ratio_offset_vel_line_shape(vel_offset,vel_sigma_OIII_br):
    ratio = (vel_offset/vel_sigma_OIII_br)
    return ratio

def line_shape_par(z,wav_min,wav_max,cor_popt):
    c = 3*(10**5)
    wave = np.arange(wav_min,wav_max,0.02) 
    cumsum = np.cumsum(OIII_wo_cont(wave,*cor_popt))
   # print (cumsum)
    norm_sum=cumsum/cumsum[-1]
    
    select = (norm_sum>0.1) & (norm_sum<0.9)
    w80_spec = wave[select][-1]-wave[select][0]
 
    select =  (norm_sum>0.05) & (norm_sum<0.5)
    v5_spec = wave[select][0]-wave[select][-1]

    [w80,v5] = [((w80_spec)/5006.8)*(c/(1+z)),((v5_spec)/5006.8)*(c/(1+z))]
    return w80,v5


def blr_sigma(obj,z,mini_wave,amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2):
    c = 300000
    k = 1+z
    m_0 = np.sum(double_BLR(mini_wave,amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2))
    m_1 = np.sum(mini_wave*double_BLR(mini_wave,amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2))/m_0
    m_2 = np.sqrt(np.sum(double_BLR(mini_wave,amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2)*(mini_wave-m_1)**2)/m_0)
    sigma_Hb_blr = c*(m_2)/m_1
    return sigma_Hb_blr

def blr_FWHM(obj,z,mini_wave,amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2):
    c = 300000
    k = 1+z
    (wav,dat) = (mini_wave,double_BLR(mini_wave,amp_Hb1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,vel_Hb2,vel_sigma_Hb2))
    amp = np.max(dat)
    select = (dat>amp/2) & (dat<amp)
    print (np.shape(wav[select]))
    fwhm = wav[select][-1] - wav[select][0]
    fwhm = (fwhm/4861.33)*(c/(1+z))
    return fwhm

def par5100(z,m,c,wavelength=5100):
    k = 1 + z
    flux_den = (m*wavelength*k/1000.0) + c
    flux = wavelength*flux_den*(10**(-16))
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
    D_L = cosmo.luminosity_distance(z)
    dist = D_L.to(u.cm)
    log_lum = np.log10(4*np.pi)+2*np.log10(dist.value)+np.log10(flux)
    lum = 10**log_lum
    return flux_den,flux,lum
    
def log_bolometric_luminosity(lum):
    L = 9.0*lum
    log_L = np.log10(L)
    return log_L

def logMBH(Hb_fwhm,lum_5100):
    logMBH=np.log10((Hb_fwhm/1000.)**2*(lum_5100/1.e44)**0.5)+6.91 
    return logMBH

def edd_lum(log_MBH):
    lum = 1.25*(10**38)*(10**log_MBH)
    return lum

def flux_lum_OIII_spectro(z,amp_OIII5007,amp_OIII5007_br,sigma_OIII_5007,sigma_OIII_5007_br):
    flux_5007_core = amp_OIII5007*(np.sqrt(2.*np.pi)*np.fabs(sigma_OIII_5007))
    flux_5007_wing = amp_OIII5007_br*(np.sqrt(2.*np.pi)*np.fabs(sigma_OIII_5007_br)) 
    flux = (flux_5007_core + flux_5007_wing)*(10**(-16))
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
    D_L = cosmo.luminosity_distance(z)
    dist = D_L.to(u.cm)
    log_lum = np.log10(4*np.pi)+2*np.log10(dist.value)+np.log10(flux)
    return flux,log_lum
     
def par_tab(obj,kin_par_err,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    column_names={'w80_err':0,'log_Hb_lum_err':1,'log_L_OIII_err':2}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[kin_par_err[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hduk = fits.BinTableHDU.from_columns(coldefs)
    hduk.writeto('%s/%s/%s.nonfixkin_kinparerr.fits'%(destination_path_cube,obj,obj),overwrite=True)   
    
def source_par_tab(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    t1 = Table.read('%s/%s/source_%s.fits'%(destination_path_cube,obj,obj),format='fits')
    t2 = Table.read('%s/%s/%s.fixkin_kinparerr.fits'%(destination_path_cube,obj,obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s/%s/%s.nonfixkin_kinparerr.fits'%(destination_path_cube,obj,obj),overwrite=True)


In [5]:
def algorithm_script(obj,z,broad2,p_init,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    print ('%s'%(obj))
    (mini_cube,mini_err,mini_wave,mini_header) = loadCube('%s/%s/%s.3_arcsec_minicube.fits'%(destination_path_cube,obj,obj))
    (int_spectrum,int_error) = int_spec(mini_cube,mini_err)
    kin_par_len = 3
    MC_loops = 100
    kin_par_MC = np.zeros((kin_par_len,MC_loops))
    
    for l in range(MC_loops):
        iter_spectrum = np.random.normal(int_spectrum,int_error)
        factor = scale_factor_spectrum(z,iter_spectrum,int_error,mini_wave)
        int_err = int_error*factor
        #print (factor)
        output_par = aperture_spec_fit(iter_spectrum,int_err,mini_wave,p_init,broad2,MC_loops=100)
        [amp_Hb,amp_OIII5007,vel_OIII,vel_sigma_OIII,amp_Hb_br,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br,amp_Hb1,amp_Fe5018_1,vel_Hb1,vel_sigma_Hb1,amp_Hb2,amp_Fe5018_2,vel_Hb2,vel_sigma_Hb2,m,c,v_out] = output_par
        
        sigma_eff_Hb1 = line_width_recons(vel_sigma_Hb1,rest_line=4861.33,inst_res_fwhm=0)
        sigma_eff_Hb2 = line_width_recons(vel_sigma_Hb2,rest_line=4861.33,inst_res_fwhm=0) 
        v_out = vel_OIII - vel_OIII_br
        vel_offset = v_out
        flux = flux_blr(amp_Hb1,amp_Hb2,sigma_eff_Hb1,sigma_eff_Hb2,broad2)
        flux_FeII = flux_FeII_5018(amp_Fe5018_1,amp_Fe5018_2,sigma_eff_Hb1,sigma_eff_Hb2,broad2)
        Hb_lum = Hb_blr_lum(z,flux)
              
        cor_popt = np.array([amp_OIII5007,amp_OIII5007_br,vel_OIII,vel_sigma_OIII,vel_OIII_br,vel_sigma_OIII_br])
        (wav_min,wav_max) = wavlim(vel_OIII,vel_OIII_br)
        (w80,v5) = line_shape_par(z,wav_min,wav_max,cor_popt)
        kin_param = [w80,Hb_lum]
        kin_par_MC[:,l] = kin_param
    kin_par_err = np.std(kin_par_MC,1)
    print (kin_par_err)
    par_tab(obj,kin_par_err)
    source_par_tab(obj)

In [ ]:
z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

objs = z.keys()

broad2= {'HE0021-1810':False,'HE0021-1819':False,'HE0040-1105':False,'HE0108-4743':True,'HE0114-0015':False
        ,'HE0119-0118':True,'HE0212-0059':True,'HE0224-2834':False,'HE0227-0913':True,'HE0232-0900':False
        ,'HE0253-1641':True,'HE0345+0056':True,'HE0351+0240':True,'HE0412-0803':False,'HE0429-0247':True
        ,'HE0433-1028':True,'HE0853+0102':True,'HE0934+0119':True,'HE1011-0403':True,'HE1017-0305':False
        ,'HE1029-1831':True,'HE1107-0813':True,'HE1108-2813':False,'HE1126-0407':True,'HE1237-0504':False
        ,'HE1248-1356':False,'HE1330-1013':True,'HE1353-1917':True,'HE1417-0909':False,'HE2128-0221':False
        ,'HE2211-3903':False,'HE2222-0026':True,'HE2233+0124':True,'HE2302-0857':False}

p_init= {'HE0021-1810':[0.1,1.5248,16056,50.0,1.0,3.0,15956,100.0,1.0,1.0,16056,1000.0,0,0,16056,1000.0,-0.001,2.0]
        ,'HE0021-1819':[1,12.5,15959,50.0,3,3,15759,200.0,2,2,15959,1000.0,0,0,15959,1000.0,-0.001,0.1]
        ,'HE0040-1105':[1.139,1.5,12507.0,50.0,1.0,3.0,12307.0,200.0,1.0,1.0,12507,1000.0,0,0,12507,1000.0,-0.001,2.0]
        ,'HE0108-4743':[1.7,4.5,7160.0,40.0,0.12,2.5,7100.0,120.0,1.0,1.0,6960,1300.0,1.0,1.0,7190,430.0,-0.001,2.0]
        ,'HE0114-0015':[0.1,1.5,13680,50.0,1.0,3.0,13480,100.0,1.0,1.0,13680,1000.0,0,0,13680,1000.0,-0.001,0.3]
        ,'HE0119-0118':[1,12.5,16302,50.0,3,3,16102,100.0,2,2,16302,1000.0,1,1,16302,2500.0,-0.001,0.1]
        ,'HE0212-0059':[0.1,1.0,7930,170.0,0.1,0.1,7630,450.0,1.0,1.0,7620,4000.0,0.1,0.1,4650,1090.0,-0.001,2.0]
        ,'HE0224-2834':[0.2,2.5,17940,50.0,0.3,0.3,17640,200.0,0.1,0.1,17940,1500.0,0,0,17940,1500.0,-0.001,0.1]
        ,'HE0227-0913':[3,6,4785,150,2,7,4385,300.0,7,1.8,4935,400.0,3,0.5,4735,1500.0,-0.3,4.7]
        ,'HE0232-0900':[2,40,12942,50.0,5,5,12742,100.0,5,1,12942,1000.0,0,0,12942,1000.0,-0.001,0.1]                                     
        ,'HE0253-1641':[2,10,9476,110.0,1,6,9176,370.0,1,1,9476,1500.0,0.5,0.5,9476,550.0,-0.001,0.5]
        ,'HE0345+0056':[0.5,20,9300,150,5,10,9000,300,9,1.8,9300,400,30,2.5,9300,1500,0.3,7.1]
        ,'HE0351+0240':[2.2,37,10600,77,1.1,1.5,10618,277,1.5,0.07,11580,320,3,0.35,10650,1344,-0.3,3]                                                    
        ,'HE0412-0803':[1.139,1.5,11448,50.0,0.1,0.3,11248,100.0,0.1,0.1,11448,1000.0,0,0,11448,1000.0,-0.001,0.5]
        ,'HE0429-0247':[0.9,8.5,12600,90,0.2,2.0,12500,220,1.7,0.07,12600,450,1.3,0.2,12600,1600,-0.5,4.0]
        ,'HE0433-1028':[0.9,4.0,10665.0,100.0,1.0,1.0,10465.0,300.0,1.0,1.0,10665,800.0,0.1,0.1,10665,1500.0,-0.1,1.0]
        ,'HE0853+0102':[0.1,0.9,15700,110.0,0.02,0.3,15670,320.0,0.1,0.1,15900,1640.0,0.0,0.0,17250,810.0,-0.001,0.1]
        ,'HE0934+0119':[0.6,1.9,15101,100.0,0.2,0.2,14901,400.0,0.3,0.1,15101,1000.0,0.1,0.1,15101,1000.0,0.001,0.7]
        ,'HE1011-0403':[0.1,0.6,17494,60.0,0.2,0.2,17244,200.0,0.3,0.1,17494,1500.0,0.1,0.1,17494,1500.0,0.001,0.7]
        ,'HE1017-0305':[0.1,1.5,14695,50,0.08,0.4,14595,200,0.7,0.09,14695,2000,0,0,0,0,-0.3,2.0]
        ,'HE1029-1831':[0.6,1.3,12114,50.0,0.2,0.2,11978,200.0,0.3,0.1,12114,1000.0,0.2,0.2,12114,1000.0,-0.001,0.2]
        ,'HE1107-0813':[0.04,0.3,17500,100,0.2,0.13,17400,480,0.9,0.13,17500,1000,0.9,0.08,16800,2000,-0.9,8]
        ,'HE1108-2813':[1.139,1.5,7200.0,50.0,1.0,3.0,7000.0,100.0,1.0,1.0,7200,1000.0,0,0,7200,1000.0,-0.001,2.0]
        ,'HE1126-0407':[1,13,18200,100,2.5,3,18000,300,10,1.3,18200,800,7,1.8,18200,1600,-0.9,15]
        ,'HE1237-0504':[1,10,2700.0,80.0,0.5,2,2700.0,200,2,0.5,2700.0,1500.0,0,0,2700.0,1000.0,-0.001,0.002]
        ,'HE1248-1356':[0.1,1.5,4395.0,50.0,1.0,3.0,4195.0,100.0,1.0,1.0,4395,1000.0,0,0,0,0.0,-0.001,2.0]
        ,'HE1330-1013':[0.15,0.6,6643,90,0.05,0.15,6543,200,0.12,0.04,6643,1500,0.3,0.04,6643,500,-0.02,0.3]
        ,'HE1353-1917':[0.07,0.7,10490.0,80.0,0.39,0.05,10306.0,490.0,0.1,0.02,8600.0,1500.0,0.14,0.02,12326.0,1500.0,-0.001,0.002]
        ,'HE1417-0909':[1,12.5,13200,50.0,3,3,13000,100.0,2,2,13200,1000.0,0,0,13200,1000.0,-0.001,0.1]
        ,'HE2128-0221':[0.1,1.5,15744,50.0,1.0,3.0,15544,100.0,1.0,1.0,15744,1000.0,0,0,15744,1000.0,-0.001,2.0]
        ,'HE2211-3903':[0.6,2.4,11914,50.0,0.1,0.1,11714,100.0,0.1,0.1,11914,200.0,0,0,11914,100.0,-0.001,0.2]
        ,'HE2222-0026':[0.08,0.4,17400.0,140.0,0.04,0.05,17150.0,300.0,0.4,0.1,18500,650.0,0.02,0.01,17460,1750.0,-0.001,0.002]
        ,'HE2233+0124':[0.1,1.2,16944.0,100.0,1.0,3.0,17044.0,300.0,1.0,1.0,16944,1200.0,1.0,1.0,16944,4000.0,-0.001,2.0]
        ,'HE2302-0857':[1,8,14058,200,0.1,2,14258,300,1,0.1,14058,1000,0.1,0.1,14058,1000,0.01,0.2]}

        
objs = z.keys()

for obj in objs:
    algorithm_script(obj,z[obj],broad2[obj],p_init[obj])

HE0021-1810


/home/mainak/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:120: RuntimeWarning: invalid value encountered in log10
/home/mainak/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in log10


[2793.02590306           nan           nan]
HE0021-1819
[1.17031751e+00 4.66227544e-04 3.00406365e-03]
HE0040-1105
[9.94620203e-01 5.02776427e-04 9.02326795e-04]
HE0108-4743
[2.56106259 0.02144379 0.00522481]
HE0114-0015
[1.19023382e+01 1.13757334e-02 6.74050435e-03]
HE0119-0118
[7.73663860e+00 2.59544646e-03            nan]
HE0212-0059


In [22]:
a = [1.22,3.54,8.1,8.508,9.34]

In [24]:
b = [int(x) for x in np.round(a)]